In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("sparksql-aggregations").getOrCreate()

df = spark.read.parquet("gs://pyspark-fs-sid/bikeshare_trips")

In [2]:
df.createOrReplaceTempView("bikeshare_trips_tmp")

In [ ]:
df.show(10,False)

In [ ]:
qry = """
        select 
            start_station_id,
            extract(year from start_time) as trip_year,
            extract(month from start_time) as trip_month,
            count(distinct trip_id) as total_trips
        from 
            bikeshare_trips_tmp
        group by 
            1,2,3
        order by 
            2 desc,3,4 desc
      """

spark.sql(qry).show(truncate=False)

In [5]:
df_total_trips = spark.sql(qry)
df_total_trips.createOrReplaceTempView("aggregated_trips")

In [ ]:
spark.sql("select * from aggregated_trips limit 10").show(10)

In [8]:
sql = """
       with ranked_stations as 
       (
           select 
                start_station_id,
                trip_year,
                trip_month,
                total_trips,
                row_number()
                    over (
                        partition by trip_year,trip_month
                        order by total_trips desc
                    ) as rank
            from 
                aggregated_trips
            where 
                trip_year=2022
        )
        select * from ranked_stations where rank<=3
        order by trip_month,rank
     """

df_final_aggregated = spark.sql(sql)

In [ ]:
df_final_aggregated.write.mode("overwrite").saveAsTable("top3_ranked_stations_2022")

In [ ]:
df_final_aggregated.show()